# Task 3: Store Cleaned Data in Oracle

In [7]:
# %pip install cx_Oracle pandas
# %pip install Faker

In [2]:
import os 
import sys
import pandas as pd
import matplotlib.pyplot as plt
from google_play_scraper import Sort, reviews
import cx_Oracle
import pandas as pd
from datetime import datetime
import random
from datetime import datetime, timedelta
sys.path.append(os.path.abspath("../scripts"))
from faker import Faker
from bank_reviews_schema_oracle_data import connect_to_oracle, insert_data
import warnings
warnings.filterwarnings('ignore')

In [4]:
# %%
os.chdir("..")  # Go up a directory
#print(os.getcwd())

# %%
# print(os.getcwd())

In [7]:
faker = Faker()
banks = ['CBE', 'Dashen', 'BOA']
sentiments = ['positive', 'negative', 'neutral']

data = [
    {
        "bank_name": random.choice(banks),
        "review_text": faker.sentence(),
        "sentiment": random.choice(sentiments),
        "rating": random.randint(1, 5),
        "review_date": faker.date_between(start_date='-1y', end_date='today')
    }
    for _ in range(1000)
]

df = pd.DataFrame(data)
df.to_csv("data/oracle_cleaned_reviews.csv", index=False)

In [8]:
# ------------------------
# 📌 Function: Connect to Oracle
# ------------------------
def connect_to_oracle(user, password, dsn):
    try:
        conn = cx_Oracle.connect(user, password, dsn)
        print("Connected to Oracle")
        return conn
    except Exception as e:
        print("Oracle connection error:", e)
        return None

# ------------------------
# 📌 Function: Load CSV
# ------------------------
def load_reviews_from_csv(file_path):
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"{file_path} not found!")
    df = pd.read_csv(file_path)
    df['review_date'] = pd.to_datetime(df['review_date'])
    print(f"Loaded {len(df)} reviews")
    return df

# ------------------------
# 📌 Function: Insert Data
# ------------------------
def insert_reviews(conn, df):
    cursor = conn.cursor()
    bank_id_map = {}

    # Insert unique banks
    for bank in df['bank_name'].unique():
        cursor.execute("SELECT bank_id FROM Banks WHERE name = :1", [bank])
        result = cursor.fetchone()
        if result:
            bank_id = result[0]
        else:
            var_out = cursor.var(cx_Oracle.NUMBER)
            cursor.execute("INSERT INTO Banks (name) VALUES (:1) RETURNING bank_id INTO :2", [bank, var_out])
            bank_id = var_out.getvalue()
        bank_id_map[bank] = bank_id

    # Insert review records
    for _, row in df.iterrows():
        cursor.execute("""
            INSERT INTO Reviews (bank_id, review_text, sentiment, rating, review_date)
            VALUES (:1, :2, :3, :4, TO_DATE(:5, 'YYYY-MM-DD'))
        """, (
            bank_id_map[row['bank_name']],
            row['review_text'],
            row['sentiment'],
            int(row['rating']),
            row['review_date'].strftime('%Y-%m-%d')
        ))
    
    conn.commit()
    print(f"✅ Inserted {len(df)} reviews")


In [ ]:
# ------------------------
# 📌 Main Entry
# ------------------------
# def main():
USER = "bank_reviews"
PASSWORD = "System123#"
DSN = "localhost/XEPDB1"
CSV_PATH = "data/oracle_cleaned_reviews.csv"

df = load_reviews_from_csv(CSV_PATH)
conn = connect_to_oracle(USER, PASSWORD, DSN)

if conn:
    insert_reviews(conn, df)
    conn.close()

# if __name__ == "__main__":
    # main()


Loaded 1000 reviews
Connected to Oracle


In [5]:
df = pd.read_csv("data/oracle_cleaned_reviews.csv")
df

,bank_name,review_text,sentiment,rating,review_date
0,BOA,Investment finish table rather moment.,neutral,2,2025-01-23
1,BOA,Star leave not fire thought.,negative,1,2025-06-02
2,BOA,Within single doctor left oil watch.,negative,3,2024-08-11
3,CBE,Throw hospital if summer especially food such.,negative,1,2024-12-07
4,BOA,High through reason fly.,positive,1,2025-01-20
...,...,...,...,...,...
995,Dashen,Song lawyer probably.,positive,3,2025-05-26
996,Dashen,Physical above how PM laugh major office and.,positive,4,2024-11-12
997,Dashen,Fly product poor left ten surface.,negative,2,2025-01-22
998,CBE,Head apply nature argue.,negative,2,2024-07-13


In [7]:
print("Looking for file in:", os.getcwd())

try:
    df = pd.read_csv("data/oracle_cleaned_reviews.csv")
    print(" File loaded successfully:")
    print(df.head())
except FileNotFoundError:
    print("File not found. Please check the path and try again.")

Looking for file in: c:\Users\Belay\Customer-Experience-Analytics-for-Fintech-Apps
 File loaded successfully:
  bank_name                                     review_text sentiment  rating  \
0       BOA          Investment finish table rather moment.   neutral       2   
1       BOA                    Star leave not fire thought.  negative       1   
2       BOA            Within single doctor left oil watch.  negative       3   
3       CBE  Throw hospital if summer especially food such.  negative       1   
4       BOA                        High through reason fly.  positive       1   

  review_date  
0  2025-01-23  
1  2025-06-02  
2  2024-08-11  
3  2024-12-07  
4  2025-01-20  
